## 各軸独立制御
「モーションコントロール」(島田明 著)の各軸独立制御(位置制御)を参考に、目標関節角度$\bm{q}_r$を参照としたときのPD制御則は下記となる。
$$
\bm{u} = K_p\bm{e}+K_d\bm{\dot{e}} \tag{1}\\ 
\bm{e} = \bm{q}_r-\bm{q}
$$
$K_p$, $K_d$は比例・微分ゲインである。重力補償を含める場合、重力補償トルク$\bm{\tau}_g$を含めると制御則は下記となる。
$$
\bm{u}_g = K_p\bm{e}+K_d\bm{\dot{e}} + \bm{\tau}_g(\bm{q}) \tag{2}
$$
#### 実装メモ
* $\bm{q}_r$は`rtb`の`IK_LM()`を用いた数値解を使用した。内部ではLM法を使用している。
* $\bm{\tau}_g(\bm{q})$も`rtb`の`gravload()`を用いて算出した。
* $\bm{\dot{e}}$は後退差分を用いて数値微分した。$T$は制御周期である。
$$
\bm{\dot{e}}[k] = \frac{\bm{e}[k] - \bm{e}[k-1]}{T} \tag{3}
$$

## 各軸速度制御
「実践ロボット制御」の各軸速度制御を参考に制御則を組み立てると下記になる。
$$
\bm{u} = K_p(\bm{q}_r-\bm{q})+K_d(\bm{\dot{q}}_r-\bm{\dot{q}}) \tag{4}
$$
ここで、$\bm{\dot{q}}_r$は微分逆運動学にて求める点が、各軸独立制御と異なる。
$$
 \bm{\dot{q}}_r = J^{-1}\bm{\dot{r}}_d 
$$
#### よくわからないこと
* 微分逆運動学を用いる場合と、各軸独立制御のように位置に関する逆運動学を解くことの差は何かがわからない。





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import planner.trajectoryProfiler as traj
from motioncon import motionControllerService as mcs

def tick(self, model, data):
    gc = dyn.calcGravComp(data.qpos)
    e = self.qcmd_ - data.qpos
    de = (e - self.epre_)/self.T
    self.ie_ = self.ie_ + (e+de)*self.T/2
    u = self.kp_*e + self.kd_*de + self.ki_*self.ie_ + gc
    ## set ctrl in mujoco
    data.ctrl = u
    # update the values of previous cycle
    self.epre_ = e